In [92]:
import pandas as pd 
from datetime import datetime

In [93]:
#Function to load and prepare data from each file
def load_data(file_name, maturity_month):
    #Read Excel file, skip 5 rows that contain metadata and header 
    #and specify column names explicitly
    df = pd.read_excel(file_name, 
                       sheet_name='Worksheet', 
                       skiprows=5
                       ) 

    #Drop any empty rows
    df=df.dropna(how='all')

    #Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    df=df.sort_values('Date')

    #Select the Date, PX_LAST and YLD_YTM_MID columns
    df = df[['Date', 'PX_LAST', 'YLD_YTM_MID']]
    df.columns = ['Date', f'Price_{maturity_month}25', f'Yield_{maturity_month}25']
    df=df.reset_index(drop=True)
    return df

In [94]:
#Load data for each STRIPS security
data_feb = load_data('grid1_azfqxckx.xlsx', 'Feb') #912833LU - Feb 2025
data_may = load_data('grid1_hp3h14cj.xlsx', 'May') #912833LV - May 2025
data_aug = load_data('grid1_vmoc0stp.xlsx', 'Aug') #912833LW - Aug 2025
data_nov = load_data('grid1_3wchcjeb.xlsx', 'Nov') #912833LX - Nov 2025

In [95]:
#Combined data
combined_data = data_feb.merge(data_may, on='Date', how='outer')
combined_data = combined_data.merge(data_aug, on='Date', how='outer')
combined_data = combined_data.merge(data_nov, on='Date', how='outer')

In [96]:
#Sort by date and reset index
combined_data = combined_data.sort_values('Date').reset_index(drop=True)

In [97]:
combined_data

,Date,Price_Feb25,Yield_Feb25,Price_May25,Yield_May25,Price_Aug25,Yield_Aug25,Price_Nov25,Yield_Nov25
0,2024-01-02,95.111,4.539,94.075,4.524,93.263,4.361,92.252,4.371
1,2024-01-03,95.102,4.559,94.072,4.535,93.310,4.336,92.264,4.370
2,2024-01-04,95.087,4.585,94.013,4.592,93.193,4.423,92.163,4.437
3,2024-01-05,95.114,4.593,94.057,4.585,93.206,4.437,92.203,4.433
4,2024-01-08,95.115,4.603,94.053,4.598,93.238,4.422,92.229,4.424
...,...,...,...,...,...,...,...,...,...
276,2025-01-23,NaN,NaN,98.727,4.206,97.703,4.194,96.678,4.233
277,2025-01-24,NaN,NaN,98.764,4.195,97.730,4.207,96.720,4.222
278,2025-01-27,NaN,NaN,98.784,4.164,97.772,4.147,96.760,4.183
279,2025-01-28,NaN,NaN,98.800,4.149,97.787,4.139,96.769,4.186


In [98]:
#Forward-Fill the NaN values with last known trade price and yield 
combined_data = combined_data.ffill()

In [99]:
combined_data

,Date,Price_Feb25,Yield_Feb25,Price_May25,Yield_May25,Price_Aug25,Yield_Aug25,Price_Nov25,Yield_Nov25
0,2024-01-02,95.111,4.539,94.075,4.524,93.263,4.361,92.252,4.371
1,2024-01-03,95.102,4.559,94.072,4.535,93.310,4.336,92.264,4.370
2,2024-01-04,95.087,4.585,94.013,4.592,93.193,4.423,92.163,4.437
3,2024-01-05,95.114,4.593,94.057,4.585,93.206,4.437,92.203,4.433
4,2024-01-08,95.115,4.603,94.053,4.598,93.238,4.422,92.229,4.424
...,...,...,...,...,...,...,...,...,...
276,2025-01-23,99.743,3.796,98.727,4.206,97.703,4.194,96.678,4.233
277,2025-01-24,99.743,3.796,98.764,4.195,97.730,4.207,96.720,4.222
278,2025-01-27,99.743,3.796,98.784,4.164,97.772,4.147,96.760,4.183
279,2025-01-28,99.743,3.796,98.800,4.149,97.787,4.139,96.769,4.186


In [100]:
#Convert yields to decimal
for col in combined_data.columns:
    if col.startswith('Yield'):
        combined_data[col] = combined_data[col]/100

In [101]:
combined_data

,Date,Price_Feb25,Yield_Feb25,Price_May25,Yield_May25,Price_Aug25,Yield_Aug25,Price_Nov25,Yield_Nov25
0,2024-01-02,95.111,0.04539,94.075,0.04524,93.263,0.04361,92.252,0.04371
1,2024-01-03,95.102,0.04559,94.072,0.04535,93.310,0.04336,92.264,0.04370
2,2024-01-04,95.087,0.04585,94.013,0.04592,93.193,0.04423,92.163,0.04437
3,2024-01-05,95.114,0.04593,94.057,0.04585,93.206,0.04437,92.203,0.04433
4,2024-01-08,95.115,0.04603,94.053,0.04598,93.238,0.04422,92.229,0.04424
...,...,...,...,...,...,...,...,...,...
276,2025-01-23,99.743,0.03796,98.727,0.04206,97.703,0.04194,96.678,0.04233
277,2025-01-24,99.743,0.03796,98.764,0.04195,97.730,0.04207,96.720,0.04222
278,2025-01-27,99.743,0.03796,98.784,0.04164,97.772,0.04147,96.760,0.04183
279,2025-01-28,99.743,0.03796,98.800,0.04149,97.787,0.04139,96.769,0.04186


In [111]:
#Add Time-to-Maturity columns (in years)
feb_2025 = pd.Timestamp('2025-02-28') 
may_2025 = pd.Timestamp('2025-05-31')
aug_2025 = pd.Timestamp('2025-08-30')
nov_2025 = pd.Timestamp('2025-11-30')

combined_data['Time_Feb25'] = (feb_2025 - combined_data['Date']).dt.days/365.25
combined_data['Time_May25'] = (may_2025 - combined_data['Date']).dt.days/365.25
combined_data['Time_Aug25'] = (aug_2025 - combined_data['Date']).dt.days/365.25
combined_data['Time_Nov25'] = (nov_2025 - combined_data['Date']).dt.days/365.25


#Reorder columns
column_order = ['Date', 
                'Price_Feb25', 'Time_Feb25', 'Yield_Feb25',
                'Price_May25', 'Time_May25', 'Yield_May25',
                'Price_Aug25', 'Time_Aug25', 'Yield_Aug25',
                'Price_Nov25', 'Time_Nov25', 'Yield_Nov25']

combined_data = combined_data[column_order]

In [112]:
combined_data

,Date,Price_Feb25,Time_Feb25,Yield_Feb25,Price_May25,Time_May25,Yield_May25,Price_Aug25,Time_Aug25,Yield_Aug25,Price_Nov25,Time_Nov25,Yield_Nov25
0,2024-01-02,95.111,1.158111,0.04539,94.075,1.409993,0.04524,93.263,1.659138,0.04361,92.252,1.911020,0.04371
1,2024-01-03,95.102,1.155373,0.04559,94.072,1.407255,0.04535,93.310,1.656400,0.04336,92.264,1.908282,0.04370
2,2024-01-04,95.087,1.152635,0.04585,94.013,1.404517,0.04592,93.193,1.653662,0.04423,92.163,1.905544,0.04437
3,2024-01-05,95.114,1.149897,0.04593,94.057,1.401780,0.04585,93.206,1.650924,0.04437,92.203,1.902806,0.04433
4,2024-01-08,95.115,1.141684,0.04603,94.053,1.393566,0.04598,93.238,1.642710,0.04422,92.229,1.894593,0.04424
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2025-01-23,99.743,0.098563,0.03796,98.727,0.350445,0.04206,97.703,0.599589,0.04194,96.678,0.851472,0.04233
277,2025-01-24,99.743,0.095825,0.03796,98.764,0.347707,0.04195,97.730,0.596851,0.04207,96.720,0.848734,0.04222
278,2025-01-27,99.743,0.087611,0.03796,98.784,0.339493,0.04164,97.772,0.588638,0.04147,96.760,0.840520,0.04183
279,2025-01-28,99.743,0.084873,0.03796,98.800,0.336756,0.04149,97.787,0.585900,0.04139,96.769,0.837782,0.04186


In [ ]:
#Save to CSV
